# ECA通道注意力机制

## 1. 计算过程

* 1. 全局平均池化

    将每个通道的二维特征(H*W)压缩为1个实数，将特征图从 (B, C, H, W) ----> (B, C, 1, 1)

* 2. 计算自适应卷积核的大小

    $k = |\frac{log_2(C)}{r} + \frac{b}{r}|$ 其中C为输入的通道数，b=1， r=2

* 3. 给每个特征通道生成一个权重值

    通过两个1x1卷积层构建通道间的相关性，输出的权重值数目和输入特征图的通道数相同

* 3. 归一化权重加权到每个通道的特征上

    逐通道乘以权重系数

## 2. 代码实现

In [1]:
from torch import nn
import torch
import math

class ECABlock(nn.Module):
    """通道注意力机制

    Args:
        nn (_type_): _description_
    """

    def __init__(self, channel, gamma=2,b=1, bias=False):
        super(ECABlock, self).__init__()
        
        k = int(abs((math.log(channel, 2) + b) / gamma))
        kernel_size = k if k % 2 else k + 1
        padding=kernel_size//2
        # 先在H*W维度进行压缩，全局平均池化将每个通道平均为一个值
        # (B, C, H, W) ---- (B, C, 1, 1)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)  # 自适应全局池化

        # 利用各channel维度的相关性计算权重
        # (1, 1, C) --- sigmoid
        self.fc1 = nn.Sequential(
            nn.Conv1d(1, 1, kernel_size=kernel_size, padding=padding, bias=bias),
            nn.Sigmoid(),
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, 1, c) # (B, C, 1, 1) --- (B, 1, C)
        y_out = self.fc1(y).view(b, c, 1, 1) # (B, 1, C) --- (B, 1, C) --- (B, C, 1, 1)
        return x * y_out

## 3. 验证

In [2]:
se_block = ECABlock(64)

x = torch.randn(1, 64, 102, 102)

y = se_block.forward(x)

print(x.shape)
print(y.shape)

torch.Size([1, 64, 102, 102])
torch.Size([1, 64, 102, 102])
